## Tamr Take Home - Chris Smith

In [2]:
import pandas as pd
from tools import FuzzPipe, FuzzyUSA
from pprint import pprint

## How many distinct suppliers are there in the USA spend dataset?
- Number of initial records.
- Number of records after reducing based on exact supplier/vendor matches.
- Number of records after reducing further based on “fuzzy” matching criteria. This should
    group together records where the same supplier had slightly different names (such as
    “W.W. Grainger” and “WW Grainger” or “IBM” and “International Business Machines”).
    Some of the fuzzy matching logic might also mean matching across columns such as
    matching vendorname with vendoralternatename. Fields like phonenumber,
    streetaddress, city, state, and dunsnumber can also provide useful signals.
- Some measure(s) of accuracy with explanations.

In [6]:
# pick which file to analyze
file_name = './data/FY2021_All_Contracts_Full_20220110_4.csv'

In [7]:
df = pd.read_csv(file_name)

/Users/csmizzle/Library/Caches/pypoetry/virtualenvs/usa-spending-AGbi8R_O-py3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (5,44,63,64,71,103,123,124,153) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Which fields have suitable cardinality?
- Can we target a field that has minimal missing values but also gives us a solid identifier for each company?
- DUNS is a likely candidate of the top of my head
- Looking for good mix of cardinality and low missing values

In [8]:
counts = pd.DataFrame(df.nunique()).reset_index(drop=False).rename(columns={'index': 'col', 0: 'count'})

In [9]:
counts.sort_values(by='count', ascending=True).head(10)

,col,count
132,small_business_competitiveness_demonstration_p...,1
258,1994_land_grant_college,1
19,action_date_fiscal_year,1
261,tribal_college,1
262,alaskan_native_servicing_institution,1
207,us_federal_government,2
200,native_american_owned_business,2
199,hispanic_american_owned_business,2
198,black_american_owned_business,2
78,multiple_or_single_award_idv_code,2


In [10]:
# let's filter out boolean fields
counts = counts[counts['count'] > 2]

In [11]:
# check out top 10 percent
top_10 = counts[counts['count'] >= counts['count'].quantile(.9)]

In [12]:
top_10

,col,count
0,contract_transaction_unique_key,1000000
1,contract_award_unique_key,920736
2,award_id_piid,917094
9,federal_action_obligation,358512
10,total_dollars_obligated,377650
11,base_and_exercised_options_value,351712
12,current_total_value_of_award,373958
13,base_and_all_options_value,346799
14,potential_total_value_of_award,380517
45,recipient_duns,49826


DUNs looks intriguing!

### Which fields don't have much missing data?

In [13]:
missing_values = pd.DataFrame(df.isna().sum()).reset_index(drop=False).rename(columns={'index': 'col', 0: 'count'})
# let's filter out boolean fields as those won;t be much help for our problem set
# lets start with fields that have 0 missing values
no_missing = missing_values[missing_values['count'] == 0]

In [14]:
no_missing_low_card = pd.merge(
    left=top_10,
    right=no_missing,
    left_on='col',
    right_on='col',
    how='inner'
)

In [15]:
no_missing_low_card.sort_values(by='count_x')

,col,count_x,count_y
7,recipient_duns,49826,0
5,base_and_all_options_value,346799,0
3,federal_action_obligation,358512,0
4,total_dollars_obligated,377650,0
6,potential_total_value_of_award,380517,0
9,last_modified_date,739532,0
2,award_id_piid,917094,0
1,contract_award_unique_key,920736,0
8,usaspending_permalink,920736,0
0,contract_transaction_unique_key,1000000,0


In [16]:
duns_count = no_missing_low_card[no_missing_low_card.col == 'recipient_duns']['count_x'].values[0]

#### Looks like DUNs is an ideal field to groupby on
- No missing values
- Suitable cardinality
- Unique identifer
- We can unify records on that key with minimal data loss

Lets clean up the recipient dataset and drop any duplicates that have like string fields

Goal is to get as close DUNs unique records and then scale the analysis up with more data

Let's subset all columns with that are dealing recipients and use DUNs as our primary indentifier

In [17]:
larger_recip_data = [
    col for col in df.columns
    if str(col).startswith('recipient')
]
larger_recip_data

['recipient_duns',
 'recipient_uei',
 'recipient_name',
 'recipient_doing_business_as_name',
 'recipient_parent_duns',
 'recipient_parent_uei',
 'recipient_parent_name',
 'recipient_country_code',
 'recipient_country_name',
 'recipient_address_line_1',
 'recipient_address_line_2',
 'recipient_city_name',
 'recipient_county_name',
 'recipient_state_code',
 'recipient_state_name',
 'recipient_zip_4_code',
 'recipient_congressional_district',
 'recipient_phone_number',
 'recipient_fax_number']

In [18]:
recip_data = df[larger_recip_data]

### Workflow is broken into two main classes
These classes are logical groupings of dataframe operations using DUNs to group like records together
- FuzzPipe -> simple cleaning and deduping
    - Normalize data into string format
        - Strip trailing zeros from float from string fields
        - Dedup identical records from string fields
    - Establish a UID (in our case DUNs) and use that to find ids with with repeats values for our UID
        - In our case, we are looking for DUNs with multiple names still associated with them
    - Filter to fields with both single and multi UIDs for down stream fuzzy matching and joining into golden table
- FuzzyUSA -> Fuzzy matching based on multiple fields on our UID
    - Take data with multiple values for a UID field (DUNs)
    - Group those values into look up table to increase performance and elimiate long looping
    - Convert fields to string to be evaluated by fuzzy matching algorithm `rapidfuzz`
        - Using simple ratio for now (could test other approaches or ensemble together)
        - Simple Ratio -> ratio of characters shared between comparison
    - Score like values against first record
        - If average score is greater than 90 for subset based UID, use first record (this can be improved, works for now)
        - Generate `match_report` off of FuzzyUSA for analysis
    - Return unifed records based on evaluation

In [21]:
pipe = FuzzPipe(recip_data)

In [22]:
multi_duns, single_duns = pipe.run(
    group_id='recipient_duns',
    count_field='recipient_name'
)

[!] Original size: 1000000
[!] Dedup on string fields size: 70875
[!] Multi IDs: 32918
[!] Single IDs: 37957


In [23]:
fuzzer = FuzzyUSA(multi_duns)
deduped = fuzzer.fuzz_match(
    key_label='recipient_duns',
    fuzz_fields=list(multi_duns.columns)
)

[!] Length before resolving: 32918


100%|██████████| 11865/11865 [00:00<00:00, 65645.36it/s]

[!] Missing keys: 0
[!] Length after resolving: 11865


In [24]:
pprint(fuzzer.match_report[0:5])

[{'compare': '57799637 * QT2NRCXEU2D3 * HARTFORD PROVISION COMPANY, THE * nan '
             '* 57799637 * QT2NRCXEU2D3 * HARTFORD PROVISION COMPANY THE * USA '
             '* UNITED STATES * 625 NUTMEG RD N * nan * SOUTH WINDSOR * '
             'HARTFORD * CT * CONNECTICUT * 6074244 * nan * 80088398 * '
             '860583657 * ',
  'compared': '57799637 * QT2NRCXEU2D3 * HARTFORD PROVISION COMPANY, THE * nan '
              '* 57799637 * QT2NRCXEU2D3 * HARTFORD PROVISION COMPANY THE * '
              'USA * UNITED STATES * 625 NUTMEG RD N * nan * SOUTH WINDSOR * '
              'HARTFORD * CT * CONNECTICUT * 6074244 * 1 * 80088398 * '
              '860583657 * ',
  'match': 99.2156862745098},
 {'compare': '7914906 * KABZK8W6PQT3 * AMERISOURCEBERGEN DRUG CORPORATION * '
             'nan * 3927759 * NWEGNLYTBDW4 * AMERISOURCEBERGEN CORPORATION * '
             'USA * UNITED STATES * 1300 MORRIS DR STE 1 * nan * CHESTERBROOK '
             '* CHESTER * PA * PENNSYLVANIA * 190875559 

In [25]:
final = pd.concat([single_duns, deduped])

In [27]:
final.head()

,recipient_duns,recipient_uei,recipient_name,recipient_doing_business_as_name,recipient_parent_duns,recipient_parent_uei,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,recipient_congressional_district,recipient_phone_number,recipient_fax_number
5,963765453,E18HG4M6MA86,"HUMAN GEO GROUP, THE",nan,80921865,RJVMG1Y7QSF8,HUMAN GEO GROUP LLC THE,USA,UNITED STATES,4350 N FAIRFAX DR STE 95,nan,ARLINGTON,ARLINGTON,VA,VIRGINIA,222031673,8,7038881247,7038881247
8,197468465,EMHFNF3LKVM6,"FFF ENTERPRISES, INC",nan,197468465,EMHFNF3LKVM6,FFF ENTERPRISES INC,USA,UNITED STATES,44000 WINCHESTER RD,nan,TEMECULA,RIVERSIDE,CA,CALIFORNIA,925902578,5,95129625,9512404474
11,4449307,KNJBKL6JH7F6,"UNI-TECH COLLISION REPAIR, LLC",nan,4449307,KNJBKL6JH7F6,UNITECH COLLISION REPAIR,USA,UNITED STATES,1994 MORELAND PKWY STE 2A,nan,ANNAPOLIS,ANNE ARUNDEL,MD,MARYLAND,214013169,3,41099091,4109909101
23,25734595,YRZBPY8N5884,LESIEUR APPRAISALS,nan,25734595,YRZBPY8N5884,LESIEUR APPRAISALS,USA,UNITED STATES,301 COTTONWOOD DR,nan,LAKE HAVASU CITY,MOHAVE,AZ,ARIZONA,864037482,4,928257184,8664157595
58,168544281,PEQJVJENSYA4,HISTOSERV INC,nan,168544281,PEQJVJENSYA4,HISTOSERV INC,USA,UNITED STATES,19526 AMARANTH DR,nan,GERMANTOWN,MONTGOMERY,MD,MARYLAND,208741202,6,30197226,3019722627


In [26]:
print(f'Final records for recipients in {file_name}\nis {len(final)} compared to DUNs {duns_count} total')

Final records for recipients in ./data/FY2021_All_Contracts_Full_20220110_4.csv
is 49822 compared to DUNs 49826 total


### Next steps and comments
- From here, we can join this clean recipient data back into our larger dataset to use for further analysis. Having clean records to join and uniquely indentify recipents will allows us to pull toegether aggregate metrics using a resovled entity, giving us a more complete picture of what a given entity looks like in a dataset.
- One weakness of this approach is it's reliance on DUNs number. If a company were to misreport their DUNs or have an upstream data entry mistake where the DUNs was misassigned to a company, this approach would falter since it uses DUNs to group values together prior to matching.
- We also lose the records that get macthed upon but more work can be done to either concatinate them into multi value fields in the final result ...